In [1]:
import json
import math
from numpy.linalg import norm
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
import string
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import cohen_kappa_score as qwk
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.pipeline import Pipeline
from spell import spell_check
# from Lemma import lemmatization
from textblob import TextBlob

In [2]:
text1 = input("Text 1")
text2 = input("Text 2")

In [3]:
data = pd.DataFrame([text2])
data.columns = ['Teks 2']
data['Teks 1'] = text1

In [4]:
data

,Teks 2,Teks 1
0,Usulan perluasan posisi bagi tentara aktif di ...,"Dalam UU TNI saat ini, prajurit aktif diizinka..."


In [5]:
print("Text 1: ",text1)
print("Text 2: ", text2)

Text 1:  Dalam UU TNI saat ini, prajurit aktif diizinkan untuk menduduki jabatan di luar instansi militer, yang terbatas di sejumlah kementerian.  Yaitu Kemenko Polhukam, Kementerian Pertahanan, Sekretariat Militer Presiden, Badan Intelijen Negara, dan Badan Siber dan Sandi Negara.  Mereka juga diizinkan aktif di Lembaga Ketahanan Nasional, Dewan Pertahanan Nasional, Badan Nasional Pencarian dan Pertolongan, Badan Narkotika Nasional, serta Mahkamah Agung.  Dalam usulan revisi Pasal 47 ayat 2 UU TNI itu, jabatan prajurit aktif diperluas ke sejumlah kementerian lainnya.  Yakni, Kementerian Koordinator Bidang Kemaritiman dan Investasi, Kementerian Kelautan dan Perikanan, Staf Kepresidenan, Badan Nasional Penanggulangan Terorisme (BNPT), dan Badan Nasional Penanggulangan Bencana (BNPB).  Mereka juga disebutkan dapat menduduki jabatan di Badan Nasional Pengamanan Perbatasan, Badan Keamanan Laut, Kejaksaan Agung, dan kementerian atau lembaga yang membutuhkan tenaga dan keahlian prajurit akti

In [6]:
#--------------------------------case folding---------------------------------
def case_folding(text):
  pattern = r'[' + string.punctuation + ']'
  punct = re.sub(pattern," ",str(text))
  case_fold = punct.lower()
  return case_fold

def spellcheck(text):
  text = spell_check(text)
  return text

def tokenization(text):
    tokens = re.split(' ',text)
    return tokens

def remove_digits(text):
  text = [item for item in text if item.isalpha()]
  return text

sw= nltk.corpus.stopwords.words('indonesian')
def remove_SW(text):
   text = [item for item in text if not item in sw]
   return text

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
  text = [stemmer.stem(item) for item in text]
  return text

def lemma(text):
  text = lemmatization(text)
  return text


## Pre-Processing

### Case folding

In [7]:
caseFoldText2 = case_folding(text2)
caseFoldText1 = case_folding(text1)

print(caseFoldText2)
print(caseFoldText1)

usulan perluasan posisi bagi tentara aktif di jabatan jabatan sipil dalam revisi undang undang nomor 34 tahun 2004 tentang tentara nasional indonesia  tni   disebut aktivis  berpotensi mengembalikan peran dwifungsi abri   dwifungsi abri  angkatan bersenjata republik indonesia  adalah kewenangan yang dimiliki tentara saat presiden suharto berkuasa   selain menjaga pertahanan  tni di masa itu juga berperan dalam sektor politik sosial   para pimpinan militer kala itu menduduki posisi posisi sipil seperti bupati  gubernur  menteri hingga jabatan lainnya   fungsi tentara seperti itu kemudian disebut aktivis disalahgunakan oleh penguasa orde baru untuk menciptakan  rezim otoriter yang melemahkan demokrasi    dalam perkembangannya  dwifungsi abri kemudian dihilangkan setelah reformasi 1998   saat itu tentara dikembalikan ke ‘barak’ untuk memperkuat profesionalisme dan menjalankan tugas utamanya yaitu menjaga kedaulatan serta teritoral negara   terkait tudingan itu  kepala pusat penerangan tni

### Spell check

In [8]:
spellText2 = spell_check(caseFoldText2)
spellText1 = spell_check(caseFoldText1)

# print(spellText2)
print("Sebelum: ",text1, '\n')
print("Sesudah:", spellText1)

Sebelum:  Dalam UU TNI saat ini, prajurit aktif diizinkan untuk menduduki jabatan di luar instansi militer, yang terbatas di sejumlah kementerian.  Yaitu Kemenko Polhukam, Kementerian Pertahanan, Sekretariat Militer Presiden, Badan Intelijen Negara, dan Badan Siber dan Sandi Negara.  Mereka juga diizinkan aktif di Lembaga Ketahanan Nasional, Dewan Pertahanan Nasional, Badan Nasional Pencarian dan Pertolongan, Badan Narkotika Nasional, serta Mahkamah Agung.  Dalam usulan revisi Pasal 47 ayat 2 UU TNI itu, jabatan prajurit aktif diperluas ke sejumlah kementerian lainnya.  Yakni, Kementerian Koordinator Bidang Kemaritiman dan Investasi, Kementerian Kelautan dan Perikanan, Staf Kepresidenan, Badan Nasional Penanggulangan Terorisme (BNPT), dan Badan Nasional Penanggulangan Bencana (BNPB).  Mereka juga disebutkan dapat menduduki jabatan di Badan Nasional Pengamanan Perbatasan, Badan Keamanan Laut, Kejaksaan Agung, dan kementerian atau lembaga yang membutuhkan tenaga dan keahlian prajurit akt

### Tokenization

In [9]:
tokenText2 = tokenization(spellText2)
tokenText1 = tokenization(spellText1)
tokenText2 = remove_digits(tokenText2)
tokenText1 = remove_digits(tokenText1)

print(tokenText2)
print(tokenText1)

['usulan', 'perluasan', 'posisi', 'bagi', 'tentara', 'aktif', 'di', 'jabatan', 'jabatan', 'sipil', 'dalam', 'revisi', 'undang', 'undang', 'nomor', 'tahun', 'tentang', 'tentara', 'nasional', 'indonesia', 'tentara', 'nasional', 'indonesia', 'disebut', 'aktivis', 'berpotensi', 'mengembalikan', 'peran', 'dwifungsi', 'abri', 'dwifungsi', 'abri', 'angkatan', 'bersenjata', 'republik', 'indonesia', 'adalah', 'kewenangan', 'yang', 'dimiliki', 'tentara', 'saat', 'presiden', 'suharto', 'berkuasa', 'selain', 'menjaga', 'pertahanan', 'tentara', 'nasional', 'indonesia', 'di', 'masa', 'itu', 'juga', 'berperan', 'dalam', 'sektor', 'politik', 'sosial', 'para', 'pimpinan', 'militer', 'kala', 'itu', 'menduduki', 'posisi', 'posisi', 'sipil', 'seperti', 'bupati', 'gubernur', 'menteri', 'hingga', 'jabatan', 'lainnya', 'fungsi', 'tentara', 'seperti', 'itu', 'kemudian', 'disebut', 'aktivis', 'disalahgunakan', 'oleh', 'penguasa', 'orde', 'baru', 'untuk', 'menciptakan', 'rezim', 'otoriter', 'yang', 'melemahkan'

In [10]:
filterText2 = remove_SW(tokenText2)
filterText1 = remove_SW(tokenText1)

print(filterText2)
print(filterText1)

['usulan', 'perluasan', 'posisi', 'tentara', 'aktif', 'jabatan', 'jabatan', 'sipil', 'revisi', 'undang', 'undang', 'nomor', 'tentara', 'nasional', 'indonesia', 'tentara', 'nasional', 'indonesia', 'aktivis', 'berpotensi', 'mengembalikan', 'peran', 'dwifungsi', 'abri', 'dwifungsi', 'abri', 'angkatan', 'bersenjata', 'republik', 'indonesia', 'kewenangan', 'dimiliki', 'tentara', 'presiden', 'suharto', 'berkuasa', 'menjaga', 'pertahanan', 'tentara', 'nasional', 'indonesia', 'berperan', 'sektor', 'politik', 'sosial', 'pimpinan', 'militer', 'menduduki', 'posisi', 'posisi', 'sipil', 'bupati', 'gubernur', 'menteri', 'jabatan', 'fungsi', 'tentara', 'aktivis', 'disalahgunakan', 'penguasa', 'orde', 'menciptakan', 'rezim', 'otoriter', 'melemahkan', 'demokrasi', 'perkembangannya', 'dwifungsi', 'abri', 'dihilangkan', 'reformasi', 'tentara', 'dikembalikan', 'kah', 'memperkuat', 'profesionalisme', 'menjalankan', 'tugas', 'utamanya', 'menjaga', 'kedaulatan', 'teritoral', 'negara', 'terkait', 'tudingan', 

### Stemming

In [11]:
stemmedText2 = stemming(filterText2)
stemmedText1 = stemming(filterText1)

print(stemmedText2)
print(stemmedText1)

['usul', 'luas', 'posisi', 'tentara', 'aktif', 'jabat', 'jabat', 'sipil', 'revisi', 'undang', 'undang', 'nomor', 'tentara', 'nasional', 'indonesia', 'tentara', 'nasional', 'indonesia', 'aktivis', 'potensi', 'kembali', 'peran', 'dwifungsi', 'abri', 'dwifungsi', 'abri', 'angkat', 'senjata', 'republik', 'indonesia', 'wenang', 'milik', 'tentara', 'presiden', 'suharto', 'kuasa', 'jaga', 'tahan', 'tentara', 'nasional', 'indonesia', 'peran', 'sektor', 'politik', 'sosial', 'pimpin', 'militer', 'duduk', 'posisi', 'posisi', 'sipil', 'bupati', 'gubernur', 'menteri', 'jabat', 'fungsi', 'tentara', 'aktivis', 'disalahgunakan', 'kuasa', 'orde', 'cipta', 'rezim', 'otoriter', 'lemah', 'demokrasi', 'kembang', 'dwifungsi', 'abri', 'hilang', 'reformasi', 'tentara', 'kembali', 'kah', 'kuat', 'profesionalisme', 'jalan', 'tugas', 'utama', 'jaga', 'daulat', 'teritoral', 'negara', 'kait', 'tuding', 'kepala', 'pusat', 'terang', 'tentara', 'nasional', 'indonesia', 'laksamana', 'muda', 'julius', 'widjojono', 'ide

## TF-IDF + Cosine + Score

In [12]:
def term(text1, text2):
    for i in text1:
        if i == '':
            text1.remove('')
    for i in text2:
        if i == '':
            text2.remove('')
            
    BoWtext1 = set(text1)
    BoWtext2 = set(text2)


    uniqueWords = BoWtext1.union(BoWtext2)
    # print(uniqueWords)

    wordsLengthText1 = dict.fromkeys(uniqueWords, 0)
    for word in text1:
        wordsLengthText1[word] += 1

    wordsLengthText2 = dict.fromkeys(uniqueWords, 0)
    for word in text2:
        wordsLengthText2[word] += 1
    
    # print('Unique words', wordsLengthText2)
        
    term = pd.DataFrame([wordsLengthText1, wordsLengthText2])
    term = term.transpose()
    term.columns = ['TFext1','TFext2']

    # display(term)

    DFtext1 = dict.fromkeys(uniqueWords, 0)
    for word in BoWtext1:
        DFtext1[word] += 1

    DFtext2= dict.fromkeys(uniqueWords, 0)
    for word in BoWtext2:
        DFtext2[word] += 1
    
    term['DFext1'] = DFtext1.values()
    term['DFext2'] = DFtext2.values()
    
    DF = []
    for i in range(len(uniqueWords)):
        DF.append(term['DFext1'][i] + term['DFext2'][i])
    term['DF'] = DF
    # display(term)
    
    idfDict = []
    
    for i in range(len(term['DF'])):
        idfDict.append(math.log10(2 / (term['DF'][i] + 1)))
        # print(idfDict)
    term['IDF'] = idfDict

    # display(term)

    tfidfText1 = []
    tfidfText2 = []
    for i in range(len(uniqueWords)):
        tfidfText1.append(term['TFext1'][i]*term['IDF'][i])
        tfidfText2.append(term['TFext2'][i]*term['IDF'][i])

    term['TF IDF 1'] = np.array(tfidfText1)
    term['TF IDF 2'] = np.array(tfidfText2)

    cosine = np.dot(tfidfText1,tfidfText2)/(np.linalg.norm(tfidfText1)*np.linalg.norm(tfidfText2))
    
    if math.isnan(cosine):
        cosine = 0

    print(cosine)
    print(len(term))
    # print('Skor: ', round((cosine*100),2))
    # print(len(term))

    return term

In [13]:
term(tokenization(text1), tokenization(text2))

0.7457572709444027
354


,TFext1,TFext2,DFext1,DFext2,DF,IDF,TF IDF 1,TF IDF 2
Yaitu,1,0,1,0,1,0.000000,0.000000,0.000000
"Kepresidenan,",1,0,1,0,1,0.000000,0.000000,0.000000
masuk,1,0,1,0,1,0.000000,0.000000,0.000000
sipil,2,3,1,1,2,-0.176091,-0.352183,-0.528274
menambahkan,1,0,1,0,1,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
itu,2,6,1,1,2,-0.176091,-0.352183,-1.056548
ideal,0,1,0,1,1,0.000000,0.000000,0.000000
penambahan,1,0,1,0,1,0.000000,0.000000,0.000000
rangkuman,0,1,0,1,1,0.000000,0.000000,0.000000


In [14]:
term(stemmedText2, stemmedText1)

0.7433444284885165
188


,TFext1,TFext2,DFext1,DFext2,DF,IDF,TF IDF 1,TF IDF 2
bnpb,0,1,0,1,1,0.000000,0.000000,0.000000
lemah,1,0,1,0,1,0.000000,0.000000,0.000000
masuk,0,1,0,1,1,0.000000,0.000000,0.000000
sipil,4,5,1,1,2,-0.176091,-0.704365,-0.880456
menang,0,1,0,1,1,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
libat,0,1,0,1,1,0.000000,0.000000,0.000000
reformasi,2,1,1,1,2,-0.176091,-0.352183,-0.176091
ideal,1,0,1,0,1,0.000000,0.000000,0.000000
politis,0,1,0,1,1,0.000000,0.000000,0.000000


## Norm?

In [15]:
import math
from numpy.linalg import norm

def term_ori(text1queries, text2queries):
    #--------------------term-----------------
    print(len(text1queries))
    print(len(text2queries))
    BoWtext1 = set(text1queries)
    BoWtext2 = set(text2queries)
    uniqueWords = BoWtext1.union(BoWtext2)

    #-----term frequency of word in doc-------
    wordsLengthText1 = dict.fromkeys(uniqueWords, 0)
    for word in text1queries:
        wordsLengthText1[word] += 1

    wordsLengthText2 = dict.fromkeys(uniqueWords, 0)
    for word in text2queries:
        wordsLengthText2[word] += 1
    
    # print(wordsLengthText2)
    term = pd.DataFrame([wordsLengthText1, wordsLengthText2])
    term = term.transpose()
    term.columns = ['term_q','term_ans']

    #--------------term frequency (TF)---------------
    tfQ = {}
    tfA = {}

    bagOfWordsCount = len(text1queries)
    #tf Kunci Jawaban
    for word, count in wordsLengthText1.items():
        tfQ[word] = count / float(bagOfWordsCount)

    #tf Jawaban
    for word, count in wordsLengthText2.items():
        if len(text2queries) == 0:
            tfA[word] = 0
        else:
            tfA[word] = count / float(len(text2queries))
   
    term['TFext1'] = tfQ.values()
    term['TFext2'] = tfA.values()

    DFtext1 = dict.fromkeys(uniqueWords, 0)
    for word in BoWtext1:
        DFtext1[word] += 1

    DFtext2= dict.fromkeys(uniqueWords, 0)
    for word in BoWtext2:
        DFtext2[word] += 1
    
    DF = []
    for i in range(len(uniqueWords)):
        DF.append(list(DFtext1.values())[i]+list(DFtext2.values())[i])
    term['DF'] = DF

    #-------------------IDF---------------------------
    N = len([wordsLengthText1, wordsLengthText2])
    
    # idfDict = dict.fromkeys([wordsLengthText1, wordsLengthText2][0].keys(), 0)
    # for document in [wordsLengthText1, wordsLengthText2]:
    #     for word, val in document.items():
    #         if val > 0:
    #             idfDict[word] += 1
    
    # for word, val in idfDict.items():
    #     idfDict[word] = math.log(N+1 / float(val+1))

    idf = []
    for i in range(len(DF)):
        idf.append(math.log10(N / (DF[i]+1)))
        # print(val)

    term['IDF'] = idf

    #-------------------TF-IDF---------------------
    tfidfText1 = []
    tfidfText2 = []
    
    for i in range(len(uniqueWords)):
        tfidfText1.append(list(tfQ.values())[i]*idf[i])
        tfidfText2.append(list(tfA.values())[i]*idf[i])
    
    # tfidfText1 = list(tfidfText1)
    # tfidfText2 = list(tfidfText2)
    term['TF_IDF Q'] = tfidfText1
    term['TF_IDF A'] = tfidfText2

    cosine =np.dot(tfidfText1,tfidfText2)/(np.linalg.norm(tfidfText1)*np.linalg.norm(tfidfText2))
    print(len(tfidfText2))
    print(cosine)
    print('Skor: ', round((cosine*100),2))
    global SkorSCT
    SkorSCT = str(round((cosine*100), 2))
    
    
    return term


In [16]:
term_ori(text2.split(), text1.split())

228
370
354
0.7457572709444029
Skor:  74.58


,term_q,term_ans,TFext1,TFext2,DF,IDF,TF_IDF Q,TF_IDF A
Yaitu,0,1,0.000000,0.002703,1,0.000000,0.000000,0.000000
"Kepresidenan,",0,1,0.000000,0.002703,1,0.000000,0.000000,0.000000
masuk,0,1,0.000000,0.002703,1,0.000000,0.000000,0.000000
sipil,3,2,0.013158,0.005405,2,-0.176091,-0.002317,-0.000952
menambahkan,0,1,0.000000,0.002703,1,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
itu,6,2,0.026316,0.005405,2,-0.176091,-0.004634,-0.000952
ideal,1,0,0.004386,0.000000,1,0.000000,0.000000,0.000000
penambahan,0,1,0.000000,0.002703,1,0.000000,0.000000,0.000000
rangkuman,1,0,0.004386,0.000000,1,0.000000,0.000000,0.000000


In [17]:
print(stemmedText2)
print(stemmedText1)

['usul', 'luas', 'posisi', 'tentara', 'aktif', 'jabat', 'jabat', 'sipil', 'revisi', 'undang', 'undang', 'nomor', 'tentara', 'nasional', 'indonesia', 'tentara', 'nasional', 'indonesia', 'aktivis', 'potensi', 'kembali', 'peran', 'dwifungsi', 'abri', 'dwifungsi', 'abri', 'angkat', 'senjata', 'republik', 'indonesia', 'wenang', 'milik', 'tentara', 'presiden', 'suharto', 'kuasa', 'jaga', 'tahan', 'tentara', 'nasional', 'indonesia', 'peran', 'sektor', 'politik', 'sosial', 'pimpin', 'militer', 'duduk', 'posisi', 'posisi', 'sipil', 'bupati', 'gubernur', 'menteri', 'jabat', 'fungsi', 'tentara', 'aktivis', 'disalahgunakan', 'kuasa', 'orde', 'cipta', 'rezim', 'otoriter', 'lemah', 'demokrasi', 'kembang', 'dwifungsi', 'abri', 'hilang', 'reformasi', 'tentara', 'kembali', 'kah', 'kuat', 'profesionalisme', 'jalan', 'tugas', 'utama', 'jaga', 'daulat', 'teritoral', 'negara', 'kait', 'tuding', 'kepala', 'pusat', 'terang', 'tentara', 'nasional', 'indonesia', 'laksamana', 'muda', 'julius', 'widjojono', 'ide

In [18]:
term_ori(stemmedText2, stemmedText1)

173
242
188
0.7433444284885167
Skor:  74.33


,term_q,term_ans,TFext1,TFext2,DF,IDF,TF_IDF Q,TF_IDF A
bnpb,0,1,0.000000,0.004132,1,0.000000,0.000000,0.000000
lemah,1,0,0.005780,0.000000,1,0.000000,0.000000,0.000000
masuk,0,1,0.000000,0.004132,1,0.000000,0.000000,0.000000
sipil,4,5,0.023121,0.020661,2,-0.176091,-0.004071,-0.003638
menang,0,1,0.000000,0.004132,1,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
libat,0,1,0.000000,0.004132,1,0.000000,0.000000,0.000000
reformasi,2,1,0.011561,0.004132,2,-0.176091,-0.002036,-0.000728
ideal,1,0,0.005780,0.000000,1,0.000000,0.000000,0.000000
politis,0,1,0.000000,0.004132,1,0.000000,0.000000,0.000000


In [19]:
newfile = open("Plagiatkah.json", "w")

newfile.write('{ "Text 1": "' + caseFoldText1 + '", \n"Tekt 2": "' +
              caseFoldText2 + '", \n"Kemiripan": "' + SkorSCT + '"}')

newfile.close()